In [2]:
import pandas as pd 
import numpy as np 
from bs4 import BeautifulSoup as bs 
import requests 
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options

url_hoteles = "https://all.accor.com/ssr/app/ibis/hotels/madrid-spain/open/index.es.shtml?compositions=1&stayplus=false&snu=false&hideWDR=false&accessibleRooms=false&hideHotelDetails=false&dateIn=2025-03-01&nights=1&destination=madrid-spain"

def scrap_info_hoteles(url):
    dictio_scrap = {
        "nombre_hotel": [],
        "estrellas": [],
        "precio_noche": []
    }

    service = Service(ChromeDriverManager().install())
    options = Options()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(service=service, options=options)

    driver.get(url)

    try:
        precios_hoteles = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "rate-details__price-wrapper")))

        for precio_hotel in precios_hoteles: 
            dictio_scrap["precio_noche"].append(precio_hotel.text.split('\n')[1].replace("€", ""))

        nombre_estrellas_hoteles = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "title")))

        for info_hotel in nombre_estrellas_hoteles: 
            dictio_scrap["nombre_hotel"].append(info_hotel.text.split("\n")[0])
            dictio_scrap["estrellas"].append(info_hotel.text.split("\n")[1].split(" ")[0])

    
    except Exception as e:
        print("No se pudo encontrar los elementos:", e)

    driver.quit()
    
    return dictio_scrap


dictio_final = scrap_info_hoteles(url_hoteles)
df_hoteles_competencia = pd.DataFrame(dictio_final)
df_hoteles_competencia

,nombre_hotel,estrellas,precio_noche
0,ibis Styles Madrid Prado,3,179
1,ibis Madrid Centro las Ventas,2,172
2,ibis budget Madrid Centro las Ventas,1,119
3,ibis budget Madrid Vallecas,1,95
4,ibis Madrid Aeropuerto Barajas,2,116
5,ibis Madrid Alcorcon Tresaguas,2,90
6,ibis budget Madrid Aeropuerto,1,97
7,ibis Madrid Alcobendas,2,83
8,ibis budget Madrid Getafe,1,99
9,ibis budget Madrid Alcorcon Móstoles,2,77
